# Hello World

We will now take a look at a first concrete example of a neural network, which makes use of the Python library Keras to learn to classify hand-written digits. The dataset we will use is the MNIST handwritten digit classification, a classic dataset in the deep learning community. You can think of "solving" MNIST as the "Hello World" of deep learning programs for computer vision.

Each image in the MNIST dataset is 28x28 and contains a centered, grayscale digit. We’ll flatten each 28x28 into a 784 dimensional vector, which we’ll use as input to our neural network. Our output will be one of 10 possible categories (0 to 9).

<img src="./resources/mnist-examples.png"  style="height: 315px"/>

Let's first install the packages we'll need. Note: we need to install TensorFlow because we're going to run Keras on a TensorFlow backend (i.e. TensorFlow will power Keras).

In [1]:
pip install keras tensorflow mnist

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached tensorflow-2.3.1-cp38-cp38-win_amd64.whl (342.5 MB)
  Using cached mnist-0.2.2-py2.py3-none-any.whl (3.5 kB)
  Using cached h5py-2.10.0-cp38-cp38-win_amd64.whl (2.5 MB)
  Using cached numpy-1.19.2-cp38-cp38-win_amd64.whl (13.0 MB)
  Using cached PyYAML-5.3.1-cp38-cp38-win_amd64.whl (219 kB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
Processing c:\users\yoriv\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501\termcolor-1.1.0-py3-none-any.whl
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Processing c:\users\yoriv\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73\wrapt-1.12.1-py3-none-any.whl
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached protobuf-3.13.0-py2.py3-none-any.whl (438 kB)
  Using cached Keras_Preproc

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.


In [20]:
pip install tensorflow-gpu

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'D:\\Users\\yoriv\\anaconda3\\envs\\gpu-test\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



## 1. Import packages and classes

We'll use tf.keras, a high-level API to build and train models in TensorFlow.

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

ModuleNotFoundError: No module named 'matplotlib'

## 2. Import the MNIST dataset

The MNIST dataset is relatively small and is used to verify that an algorithm works as expected. 

We will use 60,000 images to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the MNIST directly from TensorFlow, just import and load the data.

In [ ]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

## 3. Explore the data

Let's explore the format of the dataset before training the model. The following shows there are 60,000 images in the training set, with each image represented as 28 x 28 pixels.

In [ ]:
train_images.shape

Likewise, there are 60,000 labels in the training set.

In [ ]:
len(train_labels)

Each label is an integer between 0 and 9.

In [ ]:
train_labels

There are 10,000 images in the test set. Again, each image is represented as 28 x 28 pixels. And the test set contains 10,000 image labels.

In [ ]:
print(test_images.shape)
print(len(test_labels))

## 4. Preprocess the data

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255.

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

We scale these values to a range of 0 to 1 before feeding to the neural network model. Normalized pixel values make our network easier to train. For this, we divide the values by 255. It's important that the training set and the testing set are preprocessed in the same way. 

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

We display the first 25 images from the training set and display the class name below each image. Verify that the data is in the correct format and we're ready to build and train the network.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

## 5. Build the model

The basic building block of a neural network is a layer. Layers extract representations from the data fed into them. And, hopefully, these representations are more meaningful for the problem at hand. Most of deep learning consists of chaining together simple layers.

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

- The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a 2d-array (of 28 by 28 pixels), to a 1d-array of 28 * 28 = 784 pixels. Think of this layer as unstacking rows of pixels in the image and lining them up.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely-connected, or fully-connected, neural layers. 

- The second layer has 128 nodes (or neurons) and uses the ReLU activation function. We could explain this function but it would involve a lot of math. Just remember that ReLU is a variant of the sigmoid function.

- The third (and last) layer is a 10-node softmax layer (softmax turns arbitrary real values into probabilities). This layer returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image corresponds to the digit.

## 6. Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

- Loss function — This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
- Optimizer — This is how the model is updated based on the data it sees and its loss function. We'll stick with a pretty good default: the Adam gradient-based optimizer.
- Metrics — Used to monitor the training and testing steps. Since this is a classification problem, we'll just have Keras report on the accuracy metric.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 7. Train the model

Training a model in Keras literally consists only of calling `fit()` and specifying some parameters. There are a lot of possible parameters, but we'll only manually supply a few:

- The training data (images and labels), commonly known as X and y, respectively.
- The number of epochs (iterations over the entire dataset) to train for.

In [ ]:
model.fit(train_images, train_labels, epochs=5)

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.98 (or 98%) on the training data. This doesn’t tell us much, though - we may be overfitting. The real challenge will be seeing how our model performs on our test data.

## 8. Evaluate accuracy

Evaluating the model is pretty simple.

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

It turns out that the accuracy on the test dataset is a little less than the accuracy on the training dataset. __This gap between training accuracy and test accuracy is an example of overfitting. Overfitting is when a machine learning model performs worse on new data than on its training data.__

Thus, our model achieves a 0.0817 test loss and 97.6% test accuracy (you might get slightly other values)! Not bad for your first neural network.

## 9. Make predictions

With the model trained, we can use it to make predictions about the test images.

In [ ]:
predictions = model.predict(test_images)

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction.

In [ ]:
predictions[0]

A prediction is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different digits. We can see which digit has the highest confidence value.

In [ ]:
np.argmax(predictions[0])

So the model is most confident that this image is a seven. And we can check the image to see if this is correct.

In [ ]:
plt.figure()
plt.imshow(test_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Hooray! Well done!

## 10. Exercise

Draw the first 20 __test images__ (5 images in a row). Below the image you print the actual digit (a) and the predicted digit (p). If they are the same the textcolor is green, red otherwise. You should get something like this:

<img src="./resources/digits.png"/>

Since we achieved an accuracy of 97.6 % most of the labels should be green.

In [ ]:
# solution code


## 11. Saving the Model

Now that we have a working, trained model, we’ll save it to disk so we can load it back up anytime.

In [ ]:
model.save_weights('resources/helloworldmodel.h5')

We can now reload the trained model whenever we want by rebuilding it and loading in the saved weights.

In [ ]:
from tensorflow import keras

# build the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# load the model's saved weights
model.load_weights('resources/helloworldmodel.h5')